In [1]:

from inference.models.sam3.interactive_image_segmentation import Sam3ForInteractiveImageSegmentation


model = Sam3ForInteractiveImageSegmentation(api_key="EQ8CBtIdyAMC4KZuflxx")



sam3.__version__ 0.0.2
Enabled the use of perflib.


In [2]:
from PIL import Image


image = Image.open("./car.png")
image2 = Image.open("./traffic.jpg")



In [3]:
#without cache:
#import uuid
#model.embed_image(image2, image_id=str(uuid.uuid4()))


img_embed = model.embed_image(image)





setting image


image set None


In [4]:
from inference.core.entities.requests.sam2 import (
    Sam2EmbeddingRequest,
    Sam2InferenceRequest,
    Sam2Prompt,
    Sam2PromptSet,
    Sam2SegmentationRequest,
    Point,
    InferenceRequestImage
)

prompts = Sam2PromptSet(prompts=[
    Sam2Prompt(points=[Point(x=0.5, y=0.5, positive=True)])
])

result = model.segment_image(image=image, prompts=prompts)


Skipping the post-processing step due to the error above. You can still use SAM 3 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam3/blob/main/INSTALL.md).


In [ ]:
import numpy as np


np_image = np.array(image)

img = InferenceRequestImage(type="file", value="./car.png")

prompts = Sam2PromptSet(prompts=[
    Sam2Prompt(points=[Point(x=0.5, y=0.5, positive=True)])
])

request = Sam2SegmentationRequest(format="rle", image=img, prompts=prompts)


result = model.infer_from_request(request)

In [6]:
result

Sam2SegmentationResponse(predictions=[Sam2SegmentationPrediction(masks=[[[166, 561], [167, 561]], [[2, 561], [3, 562], [6, 562], [5, 561]], [[164, 560]], [[22, 560], [21, 561], [22, 561], [23, 560], [24, 560]], [[505, 558]], [[773, 557], [773, 558]], [[166, 557], [166, 558], [169, 558], [168, 557]], [[155, 557], [155, 558], [156, 558], [156, 557]], [[119, 557], [118, 558], [125, 558], [126, 559], [126, 560], [127, 560], [127, 559], [126, 558], [124, 558], [123, 557]], [[96, 557], [94, 559], [93, 559], [93, 560], [92, 561], [93, 562], [98, 562], [98, 561], [99, 560], [100, 560], [100, 559], [99, 558], [99, 557]], [[10, 557], [10, 560], [8, 562], [10, 562], [11, 561], [12, 561], [12, 560], [10, 558]], [[825, 553], [825, 555], [826, 554], [826, 553]], [[531, 552]], [[182, 551], [181, 552], [183, 552]], [[831, 550]], [[464, 550]], [[365, 550]], [[266, 550]], [[260, 550]], [[173, 550]], [[103, 550]], [[4, 550]], [[435, 549], [434, 550], [436, 550], [437, 551], [437, 552], [436, 553], [432, 